In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from dotenv import load_dotenv
from datetime import date
load_dotenv()

True

In [2]:
# Path to the directory with the files
#dir_path = "../../../Desktop/2023_Forecast/Input_files/Q5_inputs"
#dir_path = "/Users/mhausch/Desktop/2023_Forecast/Input_files/Q5_inputs"
dir_path = os.getenv("PATH_INPUTS")
#Define the Forecaster
Forecaster = "Country"
subset_for_dropping = "Q1 2024"
commit_message = "First submission"
submission_quarter = "Q1_2024"

In [3]:
path_data = Path(dir_path)
list_paths =list(path_data.glob("*.xlsx"))
list_file_names = [path.stem for path in list_paths]
names = [name.split("_")[1] for name in list_file_names]
country_dict = dict(zip(names, list_paths))

In [4]:
list_file_names

['5QRF_France_Q1_24',
 '5QRF_Canada_Q1_24',
 '5QRF_Spain_Q1_24',
 '5QRF_South_America_Q1_24',
 '5QRF_RestEUAA_Q1_2024',
 '5QRF_United_Kingdom_Q1_24',
 '5QRF_Italy_Q1_2024',
 '5QRF_Turkey_Q1_2024']

In [5]:
df = {}
for country, file in country_dict.items():
    temp_df= pd.read_excel(Path(dir_path)/file, header=1, index_col= 0).iloc[:19, 0:5].dropna(subset=[subset_for_dropping])
    temp_df["Entity"] = f"{country}"
    df[country] = temp_df

In [6]:
#Concat all files
df_import = pd.concat([v for k,v in df.items()]).reset_index()
df_import = df_import.rename(columns={"index":"Category"})
df_import[Forecaster] = Forecaster

In [7]:
def create_feature(df):
    if "74" in df:
        return "G360CDx"
    elif "84" in df:
        return "G360 Infinity"
    elif "Reveal" in df:
        return "Reveal"
    elif "Shield" in df:
        return "Shield"
    else:
        return "None" 

In [8]:
#Create a function which determines if the row is a volume, ASP or revenue

def create_forecast_category(df):
    if "volume" in df:
        return "Volume"
    elif "Average" in df:
        return "ASP"
    elif "quarter" in df:
        return "Revenue"
    else:
        return "None" 

In [9]:
df_import["Product"] = df_import["Category"].apply(create_feature )

In [10]:
df_import["Forecast_Category"] = df_import["Category"].apply(create_forecast_category )

In [11]:
df_import["Submission date"] = date.today()
df_import["Committ"] = commit_message
df_import = df_import.drop(["Category"], axis = 1)


In [12]:
df_import = df_import.melt(id_vars=['Entity',
 'Country',
 'Product',
 'Forecast_Category',
 'Submission date',
 'Committ'], 
 value_vars=['Q1 2024',
 'Q2 2024',
 'Q3 2024',
 'Q4 2024',
 'Q1 2025'], 
 value_name="Forecast_Value",
 var_name = "Quarter")

df_import["Submission_quarter"] = submission_quarter

In [13]:
volume_forecast = df_import[df_import["Forecast_Category"] == "Volume"]
ASP_forecast = df_import[df_import["Forecast_Category"] == "ASP"]

In [14]:
# Save the dataframes as csv-files

df_import.to_csv(f"../Data/country_forecast_{submission_quarter}.csv", index= False)
volume_forecast.to_csv(f"../Data/volume_forecast_{submission_quarter}.csv", index= False)
ASP_forecast.to_csv(f"../Data/ASP_forecast_{submission_quarter}.csv", index= False)


In [15]:
volume_forecast

,Entity,Country,Product,Forecast_Category,Submission date,Committ,Quarter,Forecast_Value,Submission_quarter
0,France,Country,G360CDx,Volume,2023-12-09,First submission,Q1 2024,50.0,Q1_2024
3,France,Country,G360 Infinity,Volume,2023-12-09,First submission,Q1 2024,0.0,Q1_2024
6,France,Country,Reveal,Volume,2023-12-09,First submission,Q1 2024,0.0,Q1_2024
9,France,Country,Shield,Volume,2023-12-09,First submission,Q1 2024,0.0,Q1_2024
12,Canada,Country,G360CDx,Volume,2023-12-09,First submission,Q1 2024,30.0,Q1_2024
...,...,...,...,...,...,...,...,...,...
465,Italy,Country,Shield,Volume,2023-12-09,First submission,Q1 2025,NaN,Q1_2024
468,Turkey,Country,G360CDx,Volume,2023-12-09,First submission,Q1 2025,NaN,Q1_2024
471,Turkey,Country,G360 Infinity,Volume,2023-12-09,First submission,Q1 2025,NaN,Q1_2024
474,Turkey,Country,Reveal,Volume,2023-12-09,First submission,Q1 2025,NaN,Q1_2024
